In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')

In [3]:
f = x*x*x*y + y + 2

In [4]:
sess = tf.Session()

In [7]:
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

In [9]:
print(result)

114


In [10]:
sess.close()

In [11]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
print(result)

114


In [12]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
result

114

In [13]:
sess = tf.InteractiveSession()
init.run()
f.eval()

114

In [14]:
sess.close()

In [15]:
sess = tf.InteractiveSession()

In [16]:
w = tf.constant(4)

In [17]:
w.eval()

4

In [125]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [126]:
data = fetch_california_housing()

In [20]:
m, n = data.data.shape

In [31]:
data.data.shape

(20640, 8)

In [127]:
datax = np.c_[np.ones((m, 1)), data.data]

In [139]:
datax.shape

(20640, 9)

In [128]:
datay = data.target

In [77]:
a = [1, 2, 3, 4, 5, 6]
np.reshape(a, (1, 1, -1))

array([[[1, 2, 3, 4, 5, 6]]])

In [54]:
tf.reset_default_graph()

In [55]:
X = tf.constant(datax, dtype=tf.float32, name='X')
y = tf.constant(datay.reshape(-1, 1), dtype=tf.float32, name='y')

In [56]:
y.shape

TensorShape([Dimension(20640), Dimension(1)])

In [57]:
XT = tf.transpose(X)

In [58]:
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [59]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [60]:
theta_value

array([[ -3.74651413e+01],
       [  4.35734153e-01],
       [  9.33829229e-03],
       [ -1.06622010e-01],
       [  6.44106984e-01],
       [ -4.25131839e-06],
       [ -3.77322501e-03],
       [ -4.26648885e-01],
       [ -4.40514028e-01]], dtype=float32)

In [129]:
from sklearn.preprocessing import StandardScaler

In [130]:
sc = StandardScaler()

In [131]:
scdataX = sc.fit_transform(datax)

In [140]:
n_epochs = 1000
learning_rate = 0.01

In [88]:
tf.reset_default_graph()

In [89]:
X = tf.constant(scdataX, dtype=tf.float32, name="X")
y = tf.constant(datay.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1, 1), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(n_epochs):
        mseval, _ = sess.run([mse, training_op])
        if i % 100 == 0:
            print(mseval)
        
    best_theta = theta.eval()
    
print(best_theta)

6.92368
4.81152
4.80418
4.80339
4.80328
4.80326
4.80325
4.80325
4.80325
4.80325
[[ 0.02522969]
 [ 0.82964069]
 [ 0.11875563]
 [-0.26556727]
 [ 0.30572966]
 [-0.00450184]
 [-0.03932706]
 [-0.8998366 ]
 [-0.87049443]]


====================== Making tensorflow compute gradients ===========

In [95]:
tf.reset_default_graph()
X = tf.constant(scdataX, dtype=tf.float32, name='X')
y = tf.constant(datay.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name='theta')
error = tf.matmul(X, theta) - y
mse = tf.reduce_mean(tf.square(error))
gradient = tf.gradients(mse, [theta])[0]
update = tf.assign(theta, theta - learning_rate * gradient)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(n_epochs + 1):
        mseerr, _ = sess.run([mse, update])
        if i % 100 == 0:
            print(mseerr)
        
    best_theta = theta.eval()

print(best_theta)

9.11545
4.81321
4.80456
4.80345
4.80329
4.80326
4.80325
4.80325
4.80325
4.80325
4.80325
[[ 0.22878122]
 [ 0.82964528]
 [ 0.11875647]
 [-0.26557595]
 [ 0.30573681]
 [-0.00450158]
 [-0.03932723]
 [-0.89982617]
 [-0.87048453]]


In [106]:
scdataX[0 * batch_size:(0+1)*batch_size].shape

(100, 9)

In [154]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [155]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scdataX[indices] # not shown
    y_batch = datay.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

In [156]:
tf.reset_default_graph()
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()

with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()    

file_writer.close()
print(best_theta)

[[ 0.90454292]
 [ 0.82172459]
 [ 0.11330826]
 [-0.21355608]
 [ 0.32225591]
 [-0.00730817]
 [ 0.00515021]
 [-0.8838433 ]
 [-0.85685891]]
